# Asset Hierarchy Migration

**Prerequisite**: Python Competency

In this tutorial, we will show you how to migrate an asset hierarchy to a data model representing the same hierarchy in CDF.

In [24]:
import os
from cognite.neat.graph import NeatGraphStore
from cognite.client import CogniteClient

In [19]:
from dotenv import load_dotenv
from pathlib import Path

load_dotenv(Path(".").resolve().parent.parent / ".env")

True

In [25]:
client = CogniteClient.default_oauth_client_credentials(
    project=os.environ["CDF_PROJECT"],
    cdf_cluster=os.environ["CDF_CLUSTER"],
    tenant_id=os.environ["IDP_TENANT_ID"],
    client_id=os.environ["IDP_CLIENT_ID"],
    client_secret=os.environ["IDP_CLIENT_SECRET"],
)

In [4]:
store = NeatGraphStore.from_memory_store()

In [9]:
from cognite.neat.graph.extractors import AssetsExtractor

In [23]:
extractor = AssetsExtractor.from_hierarchy(client, root_asset_external_id="lift_pump_stations:root") 

In [26]:
store.write(extractor)

In [27]:
from cognite.neat.rules.importers import InferenceImporter

In [28]:
importer = InferenceImporter.from_graph_store(store)

In [31]:
rules, issues = importer.to_rules()

In [33]:
rules.classes

,class_,reference,match_type,comment
0,inferred:Asset,http://purl.org/cognite/neat#Asset,exact,"Inferred from knowledge graph, where this clas..."


In [65]:
# Multitypes not supported by DMS
rules.properties.data[24].value_type = rules.properties.data[24].value_type.types[1]

In [80]:
rules.properties.data[0].model_dump()

{'class_': 'inferred:Asset',
 'property_': 'name',
 'name': None,
 'description': None,
 'value_type': 'string',
 'min_count': None,
 'max_count': 1,
 'default': None,
 'reference': 'http://purl.org/cognite/neat#name',
 'match_type': None,
 'transformation': {'traversal': {'class_': {'prefix': 'neat',
    'suffix': 'Asset',
    'version': None,
    'name': None,
    'description': None}}},
 'comment': 'Class <Asset> has property <name> with value type <string> which occurs <245> times in the graph',
 'inherited': False}

In [79]:
rules.properties.data[0]

InformationProperty(validators_to_skip=set(), class_=class(prefix=inferred,suffix=Asset), property_='name', name=None, description=None, value_type=String(), min_count=None, max_count=1, default=None, reference=Url('http://purl.org/cognite/neat#name'), match_type=None, transformation=RDFPath(traversal=SingleProperty(class_=neat:Asset, property=neat:name)), comment='Class <Asset> has property <name> with value type <string> which occurs <245> times in the graph', inherited=False)

In [45]:
store.add_rules(rules)

In [48]:
from cognite.neat.rules import exporters

In [49]:
dms_exporter = exporters.DMSExporter()

In [68]:
for uploaded in dms_exporter.export_to_cdf(rules, client, dry_run=False):
    print(uploaded.as_report_str())

C:\Users\AndersAlbert\AppData\Local\pypoetry\Cache\virtualenvs\cognite-neat-WszCo0Uu-py3.12\Lib\site-packages\pydantic\main.py:176: MoreThanOneNonAlphanumericCharacterWarning: ('property', 'Shape__Length')
  self.__pydantic_validator__.validate_python(data, self_instance=self)


Spaces: created 1
Containers: created 1
Views: created 1
Data_Models: created 1


In [69]:
schema = dms_exporter.export(rules)

C:\Users\AndersAlbert\AppData\Local\pypoetry\Cache\virtualenvs\cognite-neat-WszCo0Uu-py3.12\Lib\site-packages\pydantic\main.py:176: MoreThanOneNonAlphanumericCharacterWarning: ('property', 'Shape__Length')
  self.__pydantic_validator__.validate_python(data, self_instance=self)


In [76]:
for id_, cont in schema.containers.items():
    break

In [78]:
cont.properties

{'name': ContainerProperty(type=Text(is_list=False, collation='ucs_basic'), nullable=False, auto_increment=False, name=None, default_value=None, description=None),
 'external_id': ContainerProperty(type=Text(is_list=False, collation='ucs_basic'), nullable=False, auto_increment=False, name=None, default_value=None, description=None),
 'created_time': ContainerProperty(type=Timestamp(is_list=False), nullable=False, auto_increment=False, name=None, default_value=None, description=None),
 'last_updated_time': ContainerProperty(type=Timestamp(is_list=False), nullable=False, auto_increment=False, name=None, default_value=None, description=None),
 'DesignPointFlowGPM': ContainerProperty(type=Float64(is_list=False, unit=None), nullable=False, auto_increment=False, name=None, default_value=None, description=None),
 'DesignPointHeadFT': ContainerProperty(type=Float64(is_list=False, unit=None), nullable=False, auto_increment=False, name=None, default_value=None, description=None),
 'Enabled': Con

In [81]:
from cognite.neat.graph.loaders import DMSLoader

,value
space,inferred
external_id,inferred_model
name,Inferred Model
description,Creator: NEAT
version,inferred
views,"[{'space': 'inferred', 'external_id': 'Asset',..."
is_global,False
last_updated_time,1970-01-01 00:00:00
created_time,1970-01-01 00:00:00


In [83]:
loader = DMSLoader(store, schema.as_read_model(), instance_space="sp_pump_station")

In [84]:
for thing in loader.load():
    break

In [86]:
thing.sources[0].properties

{'name': 'Pump 3102.090-0995',
 'external_id': '4469bba5-cad1-46a8-8647-f639bf5b31ef',
 'created_time': datetime.datetime(2024, 6, 24, 6, 24, 47, 431000, tzinfo=TzInfo(UTC)),
 'last_updated_time': datetime.datetime(2024, 6, 24, 6, 24, 47, 431000, tzinfo=TzInfo(UTC)),
 'DesignPointFlowGPM': 172.0,
 'DesignPointHeadFT': 34.4,
 'Enabled': 1.0,
 'HighHeadShutOff': 0.0,
 'LowHeadFT': 25.0,
 'LowHeadFlowGPM': 400.0,
 'PumpHP': 5.0,
 'PumpOff': 3540.02,
 'PumpOn': 3542.02,
 'Shape__Length': 8.9256133170089,
 'VFDSetting': nan,
 'parent': {'space': 'sp_pump_station',
  'externalId': 'Asset_8906381697072008'},
 'root': {'space': 'sp_pump_station', 'externalId': 'Asset_2622548696369535'},
 'description': 'Col Lift Sta. #41   Glenshire - Glenshire Subdivision, Behind 1064 Canoe Ct.',
 'FacilityID': 'CPUMP041B',
 'InstallDate': '2002/05/19 00:00:00+00',
 'LifeCycleStatus': 'I',
 'LiftStationID': 'Glenshire',
 'LocationDescription': 'Glenshire - 1064 NE Glenshire Pl., 97701',
 'Position': 'LAG1',
 